In [67]:
from pandas import DataFrame, read_json, to_datetime
import os
COLUMNS = ['date', 'open', 'high', 'low', 'close', 'volume']
DATA_DIR = '/freqtrade/user_data/data'
jsonFile = os.path.join(DATA_DIR, 'binance/BTC_USDT-12h.json')
print(jsonFile)

/freqtrade/user_data/data/binance/BTC_USDT-12h.json


In [68]:
dataframe = read_json(jsonFile, orient='values')
dataframe.columns = COLUMNS
dataframe = pairdata.astype(dtype={'open': 'float', 'high': 'float',
                                  'low': 'float', 'close': 'float', 'volume': 'float'})
dataframe['date'] = to_datetime(pairdata['date'],
                               unit='ms',
                               utc=True,
                               infer_datetime_format=True)

In [18]:
dataframe.tail(200)

,date,open,high,low,close,volume
3497,2022-06-01 00:00:00+00:00,31801.05,31982.97,31371.45,31609.16,20673.95506
3498,2022-06-01 12:00:00+00:00,31609.15,31901.34,29301.00,29805.83,82721.67876
3499,2022-06-02 00:00:00+00:00,29805.84,30210.18,29594.55,30173.67,21698.63899
3500,2022-06-02 12:00:00+00:00,30173.66,30689.00,29597.01,30452.62,35262.79029
3501,2022-06-03 00:00:00+00:00,30452.63,30699.00,29680.00,29758.19,23211.28885
...,...,...,...,...,...,...
3692,2022-09-06 12:00:00+00:00,19918.33,19993.67,18649.51,18790.61,198467.48297
3693,2022-09-07 00:00:00+00:00,18790.61,18873.17,18510.77,18739.59,125692.75807
3694,2022-09-07 12:00:00+00:00,18739.78,19464.06,18674.33,19292.84,161702.02073
3695,2022-09-08 00:00:00+00:00,19292.85,19397.71,19056.00,19311.15,113951.24811


In [69]:
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
dataframe['cci_fs'] = ta.CCI(dataframe, timeperiod=5)
dataframe['cci_ss'] = ta.CCI(dataframe, timeperiod=20)
dataframe['r_min'] =dataframe['cci_ss'].rolling(4).min()

In [70]:
    dataframe.loc[
        (
            # Signal: RSI crosses above 30
            (dataframe['cci_ss']>=-70) &
            (qtpylib.crossed_above(dataframe['cci_fs'], 130)) &  # Guard: tema below BB middle
            (dataframe['r_min'] < -70) &  # Guard: tema is raising
            (dataframe['volume'] > 0)  # Make sure Volume is not 0
        ),
        'enter_long'] = 1

In [43]:
dataframe.tail(80)

,date,open,high,low,close,volume,cci_fs,cci_ss,r_min,enter_long
3617,2022-07-31 00:00:00+00:00,23644.64,23886.92,23520.00,23801.07,52044.65632,-116.679160,85.455245,85.455245,NaN
3618,2022-07-31 12:00:00+00:00,23802.56,24194.82,23227.31,23293.32,75698.66851,-134.186734,65.954451,65.954451,NaN
3619,2022-08-01 00:00:00+00:00,23296.36,23466.55,22939.57,23227.79,59972.99796,-137.274746,37.173174,37.173174,NaN
3620,2022-08-01 12:00:00+00:00,23227.79,23509.68,22850.00,23268.01,84237.16423,-83.600802,35.545025,35.545025,NaN
3621,2022-08-02 00:00:00+00:00,23266.90,23447.62,22654.37,22882.61,65993.54384,-94.249573,16.015302,16.015302,NaN
...,...,...,...,...,...,...,...,...,...,...
3692,2022-09-06 12:00:00+00:00,19918.33,19993.67,18649.51,18790.61,198467.48297,-166.666667,-299.509408,-299.509408,NaN
3693,2022-09-07 00:00:00+00:00,18790.61,18873.17,18510.77,18739.59,125692.75807,-116.447198,-330.199269,-330.199269,NaN
3694,2022-09-07 12:00:00+00:00,18739.78,19464.06,18674.33,19292.84,161702.02073,-32.288873,-170.064579,-330.199269,NaN
3695,2022-09-08 00:00:00+00:00,19292.85,19397.71,19056.00,19311.15,113951.24811,4.688641,-121.449607,-330.199269,NaN


In [71]:
dataframe.tail(150).to_csv("output.csv")  

In [34]:
df = DataFrame({'cci': [-59.90115315,-87.33773969,-78.32844948,-67.65430566,-60.89719606,-18.75014656]})

df['min'] = df.rolling(4).min()
df.head(10)

,cci,min
0,-59.901153,NaN
1,-87.337740,NaN
2,-78.328449,NaN
3,-67.654306,-87.337740
4,-60.897196,-87.337740
5,-18.750147,-78.328449
